###

#### Load the HTML from the deck introduction page registered in the official DB of Yu-Gi-Oh! The deck I will be analyzing is a Dragon Link deck, which I use in Master Duel.
#### 遊戯王の公式DBに登録したデッキ紹介ページからHTMLを読み込みます。今回、分析するデッキはドラゴンリンクデッキで、自身がマスターデュエル(ゲーム)で使ってるデッキです。

Deck link ->

(JP)
http://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=ja

(ENG)
https://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=en

In [12]:
#imports

import requests
from bs4 import BeautifulSoup as bs4

import time
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import re

import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', ResourceWarning)

In [13]:
import pandas as pd

import openpyxl

from IPython.core.display import display

### Scrape the database website
###### Scrape Yu-gi-oh database and obtain HTML from targeted deck's website. 
###### 遊戯王データベースをスクレイピングし、対象となるデッキのウェブサイトからHTMLを取得します。

In [14]:
URL = 'http://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=ja'
res = requests.get(URL)
if res.status_code != requests.codes.ok:
    print('Error')
    exit(1)
soup = bs4(res.text)
#soup.head() #shows html

### Create a list
###### From HTML file,  create list of card names and count num.
###### 読み込んだHTMLからカード名と数を取得します。

In [15]:
deck_name = "DragonLink_PureVer1.0"
#how can i find text by multiple tag conditions??
# deck_name = soup.find('header', id= "broad_title").findChildren #デッキ名
# deck_name

num_title = soup.find('a', id= "num_total_m").text #メインデッキ合計
print(num_title)
num_total_e = soup.find('a', id= "num_total_e").text #エクストラデッキ合計
print(num_total_e)

print(num_title[10:12])
print(num_total_e[12:14])
num_of_main = int(num_title[10:12])
num_of_extra = int(num_total_e[12:14])

#http://cybrarian.blog.fc2.com/blog-entry-85.html


メインデッキ合計
41


エクストラデッキ合計
15

41
15


In [16]:
#create a list with card names and num

num_of_total = num_of_main + num_of_extra

card_list = soup.find_all('td', class_= "card_name") # デッキの全カード名をリストとして取得
print(card_list)
cardList = [] # 配列を作成
num_list = soup.find_all('td', class_= "num") # カードの枚数を取得
print(num_list)
numList = [] # 配列を作成

nm = 0
for n, m in zip(card_list,num_list):#spanタグのテキストデータを配列に格納
    card_txt = n.text.replace('\n', '')
    num_txt = m.text.strip() #カードの枚数を取得する際にエラー。空白を消去。
    nm = nm + int(num_txt)
    if nm == num_of_total:
        break
    else:
        cardList.append(card_txt)
        numList.append(num_txt)

print("カードの種類:{}".format(cardList))
# print("カードの種類:{}".format(set(cardList))) #setでも重複を削除可。
print(len(cardList))
print(numList)
print("カードの種類:{}".format(len(numList)))

[<td class="card_name">
<span>輝光竜セイファート</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?ope=2&amp;cid=14641&amp;request_locale=ja"/>
</td>, <td class="card_name">
<span>アブソルーター・ドラゴン</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?ope=2&amp;cid=14618&amp;request_locale=ja"/>
</td>, <td class="card_name">
<span>ヴァレット・トレーサー</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?ope=2&amp;cid=14615&amp;request_locale=ja"/>
</td>, <td class="card_name">
<span>ヴァレット・シンクロン</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?ope=2&amp;cid=14084&amp;request_locale=ja"/>
</td>, <td class="card_name">
<span>ヴァレット・リチャージャー</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?ope=2&amp;cid=14616&amp;request_locale=ja"/>
</td>, <td class="card_name">
<span>灰流うらら</span>
<input class="link_value" type="hidden" value="/yugiohdb/card_search.action?

### Get attribute by web scrapping
###### Now let Chrome to find each card information through web scraping! This function automatically search card name in Yu-gi-oh wiki and determine if its card types (Monster, Magic, etc...) Those values will be imported into dataframe.

###### クロームにカードの情報を探してもらいましょう！下記の関数は、遊戯王Wikiでカード名を検索しカードの種類を取得するプログラムを書いています。取得した情報はデータフレームに落とします。

In [17]:
# Creating an empty Dataframe with column names only
df = pd.DataFrame(columns=['Card_name', 'Card_type', 'n_of_card'])
# list of card types
card_type = ["融合モンスター","シンクロモンスター","エクシーズモンスター","リンクモンスター","モンスター","魔法","罠"]
# empty list for error
error_card = []

# Create function
def card_register(index_number,keyword, n_of_card):
    print(keyword+"の情報を確認中。")
    # Setting up chromedriver
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options) #hide Chrome
    # driver = webdriver.Chrome() #show Chrome
    driver.get('https://yugioh-wiki.net/')
    # time.sleep(1)  # Let the user actually see something
    search_box = driver.find_element_by_name('word')
    search_box.send_keys(keyword)
    search_box.submit()
    # time.sleep(1)  # Let the user actually see something
    try:
        page_link = driver.find_element_by_link_text('《'+keyword+'》')
    except NoSuchElementException:
        print(keyword+"はリンクが見つかりませんでした。\n ------") #to pick card that is not found
        error_card.append(keyword)
        df.loc[index_number]=[keyword, "unknown", n_of_card] #assign a card with an error
    else:
        url = page_link.get_attribute("href")
        # time.sleep(1)  # Let the user actually see something
        res = requests.get(url)
        if res.status_code != requests.codes.ok: 
            print('Error')
            exit(1)
        soup = bs4(res.text)
        tag = soup.find('span', class_= "tag").text #match with card types
        tag_split_list = tag.split()
        for tag in tag_split_list:
            if tag in card_type:
                print("カードの種類:"+tag+"\n ------")
                cardType = tag
        df.loc[index_number]=[keyword, cardType, n_of_card] #assign a card with card types
        driver.quit()

In [18]:
#enable above function with the card list
nm = 0
for card_name, n in zip(cardList,numList):
    nm += 1
    print(nm)
    card_register(nm,card_name,n)

1
輝光竜セイファートの情報を確認中。
カードの種類:モンスター
 ------
2
アブソルーター・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
3
ヴァレット・トレーサーの情報を確認中。
カードの種類:モンスター
 ------
4
ヴァレット・シンクロンの情報を確認中。
カードの種類:モンスター
 ------
5
ヴァレット・リチャージャーの情報を確認中。
カードの種類:モンスター
 ------
6
灰流うららの情報を確認中。
カードの種類:モンスター
 ------
7
原始生命態ニビルの情報を確認中。
カードの種類:モンスター
 ------
8
混源龍レヴィオニアの情報を確認中。
カードの種類:モンスター
 ------
9
暗黒竜 コラプサーペントの情報を確認中。
カードの種類:モンスター
 ------
10
輝白竜 ワイバースターの情報を確認中。
カードの種類:モンスター
 ------
11
マグナヴァレット・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
12
メタルヴァレット・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
13
増殖するGの情報を確認中。
増殖するGはリンクが見つかりませんでした。
 ------
14
ノクトビジョン・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
15
PSYフレームギア・γの情報を確認中。
PSYフレームギア・γはリンクが見つかりませんでした。
 ------
16
PSYフレーム・ドライバーの情報を確認中。
PSYフレーム・ドライバーはリンクが見つかりませんでした。
 ------
17
リボルブート・セクターの情報を確認中。
カードの種類:魔法
 ------
18
クイック・リボルブの情報を確認中。
カードの種類:魔法
 ------
19
竜の渓谷の情報を確認中。
カードの種類:魔法
 ------
20
混沌領域の情報を確認中。
カードの種類:魔法
 ------
21
サンダー・ボルトの情報を確認中。
カードの種類:魔法
 ------
22
ハーピィの羽根帚の情報を確認中。
カードの種類:魔法
 ------
23
龍の鏡の情報を確認中。
カードの種類:魔法
 ------
24
星遺物の守護竜の情報を確

In [19]:
display(df)
print("Card with errors /エラーが出たカードです。")
display(error_card)

,Card_name,Card_type,n_of_card
1,輝光竜セイファート,モンスター,1
2,アブソルーター・ドラゴン,モンスター,2
3,ヴァレット・トレーサー,モンスター,3
4,ヴァレット・シンクロン,モンスター,1
5,ヴァレット・リチャージャー,モンスター,1
6,灰流うらら,モンスター,2
7,原始生命態ニビル,モンスター,1
8,混源龍レヴィオニア,モンスター,1
9,暗黒竜 コラプサーペント,モンスター,1
10,輝白竜 ワイバースター,モンスター,2


Card with errors /エラーが出たカードです。


['増殖するG',
 'PSYフレームギア・γ',
 'PSYフレーム・ドライバー',
 'M・HERO ダーク・ロウ',
 'ヴァレルロード・S・ドラゴン',
 'ヴァレルロード・F・ドラゴン',
 'No.38 希望魁竜タイタニック・ギャラクシー',
 '琰魔竜 レッド・デーモン・アビス']

### Create a table
###### Organize Dataframe. This time, I filled missing attributes manually. Divide the table into main deck and extra deck based on card types.
###### データフレームを整理します。今回は欠落している属性を手動で補うこととします。カードの情報が記載されたテーブルをカードの種類によってメインデッキとエクストラデッキに分割する。

In [20]:
df_copy = df

#Classify error cards and cards in the wrong category.
monsters_w_error = ['増殖するG', 'PSYフレームギア・γ', 'PSYフレーム・ドライバー']
fusion_w_error = ['M・HERO ダーク・ロウ','ヴァレルロード・F・ドラゴン']
synchro_w_error = ['ヴァレルロード・S・ドラゴン','琰魔竜 レッド・デーモン・アビス','混沌魔龍 カオス・ルーラー']
xyz_w_error = ['No.38 希望魁竜タイタニック・ギャラクシー']
link_w_error = ['トロイメア・ユニコーン']

df_copy.loc[df_copy[df_copy['Card_name'].isin(monsters_w_error)].index,'Card_type'] = "モンスター"
df_copy.loc[df_copy[df_copy['Card_name'].isin(fusion_w_error)].index,'Card_type'] = "融合モンスター"
df_copy.loc[df_copy[df_copy['Card_name'].isin(synchro_w_error)].index,'Card_type'] = "シンクロモンスター"
df_copy.loc[df_copy[df_copy['Card_name'].isin(xyz_w_error)].index,'Card_type'] = "エクシーズモンスター"
df_copy.loc[df_copy[df_copy['Card_name'].isin(link_w_error)].index,'Card_type'] = "リンクモンスター"
display(df_copy)

,Card_name,Card_type,n_of_card
1,輝光竜セイファート,モンスター,1
2,アブソルーター・ドラゴン,モンスター,2
3,ヴァレット・トレーサー,モンスター,3
4,ヴァレット・シンクロン,モンスター,1
5,ヴァレット・リチャージャー,モンスター,1
6,灰流うらら,モンスター,2
7,原始生命態ニビル,モンスター,1
8,混源龍レヴィオニア,モンスター,1
9,暗黒竜 コラプサーペント,モンスター,1
10,輝白竜 ワイバースター,モンスター,2


In [21]:
#create logic of dividing a table
df_main = df_copy[df_copy['Card_type'].isin(["モンスター","魔法","罠"])]
df_extra = df_copy[df_copy['Card_type'].isin(["融合モンスター","シンクロモンスター","エクシーズモンスター","リンクモンスター"])]
print("Main Deck:")
display(df_main)
print("Extra Deck")
display(df_extra)

Main Deck:


,Card_name,Card_type,n_of_card
1,輝光竜セイファート,モンスター,1
2,アブソルーター・ドラゴン,モンスター,2
3,ヴァレット・トレーサー,モンスター,3
4,ヴァレット・シンクロン,モンスター,1
5,ヴァレット・リチャージャー,モンスター,1
6,灰流うらら,モンスター,2
7,原始生命態ニビル,モンスター,1
8,混源龍レヴィオニア,モンスター,1
9,暗黒竜 コラプサーペント,モンスター,1
10,輝白竜 ワイバースター,モンスター,2


Extra Deck


,Card_name,Card_type,n_of_card
29,M・HERO ダーク・ロウ,融合モンスター,1
30,ストライカー・ドラゴン,リンクモンスター,2
31,ヴァレルソード・ドラゴン,リンクモンスター,1
32,ヴァレルロード・S・ドラゴン,シンクロモンスター,1
33,ヴァレルロード・F・ドラゴン,融合モンスター,1
34,煉獄龍 オーガ・ドラグーン,シンクロモンスター,1
35,ツイン・トライアングル・ドラゴン,リンクモンスター,1
36,No.38 希望魁竜タイタニック・ギャラクシー,エクシーズモンスター,1
37,天球の聖刻印,リンクモンスター,1
38,ドラグニティナイト－ロムルス,リンクモンスター,1


### Export a data
###### Turn tables into csv and excel files!
###### 最後にCSV, Excelで出力しましょう！

In [22]:
df_copy.to_csv('{}_all.csv'.format(deck_name))
df_main.to_csv('{}_main.csv'.format(deck_name))
df_extra.to_csv('{}_extra.csv'.format(deck_name))

In [23]:
df_copy.to_excel('{}_all.xlsx'.format(deck_name))
df_main.to_excel('{}_main.xlsx'.format(deck_name))
df_extra.to_excel('{}_extra.xlsx'.format(deck_name))